# Frozen Lake Game using Reinforcement Learning 
### Ideas learnt:
    1. Bellman Equations
    2. Markov decision process
    3. Value iteration algorithm
    4. Optimal policy evaluation algorithm
    5. Solving equations using Linear Programming

In [2]:
import numpy as np
from scipy.optimize import linprog

# Define the environment
grid_size = 4
states = np.arange(grid_size ** 2)
print(states)
actions = {'N': -grid_size, 'E': 1, 'W': -1, 'S': grid_size}
print(actions)

# Rewards and discount factor
R = np.zeros(len(states))
R[-1] = 1  # Goal state reward
print(f"Reward: {R}\n")
lambda_ = 0.9
epsilon = 1e-3

holes = [5, 7, 11, 12]

# Initialize value function
V = np.zeros(len(states))

# Transition function
def transition(state, action):
    print(f"state: {state} and action: {action}\n")
    if state == 15:  # Goal state
        return 15
    if state in holes:
        return state
    new_state = state + actions[action]
    if new_state < 0 or new_state >= len(states) or (state % grid_size == 0 and action == 'W') or ((state + 1) % grid_size == 0 and action == 'E'):
        print(f"new state: {state}\n")
        return state  # Return to same state if out of bounds or invalid move
    print(f"new state: {new_state}\n")
    return new_state

# Value Iteration
def value_iteration(V, R, states, actions, lambda_, epsilon):
    print(f"value:{V}. \nReward: {R}.\n")
    delta = float('inf')
    while delta > epsilon:
        delta = 0
        for s in states:
            v = V[s]
            V[s] = max([R[s] + lambda_ * V[transition(s, a)] for a in actions])
            delta = max(delta, abs(v - V[s]))
    return V

# Run value iteration
V_optimal = value_iteration(V, R, states, actions, lambda_, epsilon)

print('Optimal Value Function:')
print(V_optimal)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
{'N': -4, 'E': 1, 'W': -1, 'S': 4}
Reward: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]

value:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]. 
Reward: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.].

state: 0 and action: N

new state: 0

state: 0 and action: E

new state: 1

state: 0 and action: W

new state: 0

state: 0 and action: S

new state: 4

state: 1 and action: N

new state: 1

state: 1 and action: E

new state: 2

state: 1 and action: W

new state: 0

state: 1 and action: S

new state: 5

state: 2 and action: N

new state: 2

state: 2 and action: E

new state: 3

state: 2 and action: W

new state: 1

state: 2 and action: S

new state: 6

state: 3 and action: N

new state: 3

state: 3 and action: E

new state: 3

state: 3 and action: W

new state: 2

state: 3 and action: S

new state: 7

state: 4 and action: N

new state: 0

state: 4 and action: E

new state: 5

state: 4 and action: W

new state: 4

state: 4 and act

In [3]:
# Extracting the policy from the optimal value function
def extract_policy(V, states, actions, lambda_):
    policy = {}
    for s in states:
        if s == 15:  # Goal state
            policy[s] = 'G'
            continue
        if s in holes:
            policy[s] = 'H'
            continue
        best_action = max(actions, key=lambda a: R[s] + lambda_ * V[transition(s, a)])
        policy[s] = best_action
    return policy

# Extract policy
policy_optimal = extract_policy(V_optimal, states, actions, lambda_)

print('Optimal Policy:')
for state, action in policy_optimal.items():
    print(f'State {state}: {action}')

state: 0 and action: N

new state: 0

state: 0 and action: E

new state: 1

state: 0 and action: W

new state: 0

state: 0 and action: S

new state: 4

state: 1 and action: N

new state: 1

state: 1 and action: E

new state: 2

state: 1 and action: W

new state: 0

state: 1 and action: S

new state: 5

state: 2 and action: N

new state: 2

state: 2 and action: E

new state: 3

state: 2 and action: W

new state: 1

state: 2 and action: S

new state: 6

state: 3 and action: N

new state: 3

state: 3 and action: E

new state: 3

state: 3 and action: W

new state: 2

state: 3 and action: S

new state: 7

state: 4 and action: N

new state: 0

state: 4 and action: E

new state: 5

state: 4 and action: W

new state: 4

state: 4 and action: S

new state: 8

state: 6 and action: N

new state: 2

state: 6 and action: E

new state: 7

state: 6 and action: W

new state: 5

state: 6 and action: S

new state: 10

state: 8 and action: N

new state: 4

state: 8 and action: E

new state: 9

state: 8 an

In [4]:
from pulp import *
# Linear Programming formulation
def linear_programming():
    num_states = len(states)
    num_actions = len(actions)
    lp_prob = LpProblem("FrozenLakeLP", LpMinimize)
    V = LpVariable.dicts("V", range(num_states), lowBound=0)
    
    # Objective Function: Minimize the sum of the state values
    lp_prob += lpSum([V[s] for s in range(num_states)])
    
    # Constraints based on the Bellman equation
    for s in range(num_states):
        for a in actions:
            next_state = transition(s, a)  # Get the next state based on the action
            lp_prob += V[s] >= R[s] + lambda_ * V[next_state]
    
    # Solve the LP problem
    lp_prob.solve(PULP_CBC_CMD(msg=0))
    v_values = np.array([value(V[i]) for i in range(num_states)])
    return v_values

In [5]:
# Linear Programming
print("\nLinear Programming:")
v_values = linear_programming()
print(v_values)


Linear Programming:
state: 0 and action: N

new state: 0

state: 0 and action: E

new state: 1

state: 0 and action: W

new state: 0

state: 0 and action: S

new state: 4

state: 1 and action: N

new state: 1

state: 1 and action: E

new state: 2

state: 1 and action: W

new state: 0

state: 1 and action: S

new state: 5

state: 2 and action: N

new state: 2

state: 2 and action: E

new state: 3

state: 2 and action: W

new state: 1

state: 2 and action: S

new state: 6

state: 3 and action: N

new state: 3

state: 3 and action: E

new state: 3

state: 3 and action: W

new state: 2

state: 3 and action: S

new state: 7

state: 4 and action: N

new state: 0

state: 4 and action: E

new state: 5

state: 4 and action: W

new state: 4

state: 4 and action: S

new state: 8

state: 5 and action: N

state: 5 and action: E

state: 5 and action: W

state: 5 and action: S

state: 6 and action: N

new state: 2

state: 6 and action: E

new state: 7

state: 6 and action: W

new state: 5

state: 6 

In [7]:
print(policy_optimal)

{0: 'E', 1: 'E', 2: 'S', 3: 'W', 4: 'S', 5: 'H', 6: 'S', 7: 'H', 8: 'E', 9: 'E', 10: 'S', 11: 'H', 12: 'H', 13: 'E', 14: 'E', 15: 'G'}
